# Auto-Emailer

This is an automatic emailer for unique requests based off of the `template_email` you declare below. The example is from a sponsorship email for an event called [DevOpsDays Austin](http://devopsdays.org/austin), the core idea is to send out a bunch of relitievly unique emails asking for sponsorships. This recipe is the way to get the `granite` model to do this for you.

## `imports`
This is the imports and core setup of the this recipe.

In [ ]:
!pip install --upgrade pip
!pip install ollama markdown
import csv
from ollama import chat
import smtplib, ssl
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import markdown

## `variables`
Some global variables to help you set things up. 
- The `sender_email` is the email you would like to send from.
- `mail_server` is the email server you need to connect to.
- `user` is the user if it needs to differ from the sender's email.
- `template_email` is the email or core story you want to tell; it's the overview or generic email you want to use.
- `prospectus` is the prospectus for the event, if you need one
- `date_of_event` is the date of the event so that the model can know
- `my_name` is your name
- `my_email` is your email
- `my_phone` is your phone number

In [ ]:
port = 465
password = input("SMTP password: ")
sender_email = "postmaster@asgharlabs.io"
mail_server = "smtp.mailgun.org"
user = sender_email

template_email = """
DevOpsDays Austin is in its thirteenth year, and we are one of the oldest DevOpsDays in the United States and the world. We consistently are near or at capacity on tickets sold (~320-400, depending on our venue) and have 90% or greater of attendees check in. Attendees skew toward more experienced and range from senior engineer to director to CEO, and they represent companies from stealth startups to large enterprises with sectors like finance, healthcare, and as-a-service platforms.

In our survey of attendees last year, one piece of feedback we received was literally "Would have liked more vendors to interact with," and that was for a sold-out sponsorship area. Our community appreciates our sponsors and wants to interact with you. We would love to have you.
"""

prospectus = "https://assets.devopsdays.org/events/2025/austin/devopsdays-austin-2025-sponsor-prospectus.pdf"

date_of_event = "May 1st, and 2nd, 2025"

my_name = "Jenny DevOper"
my_email = "jenny@example.com"
my_phone = "123-867-5309"

## Load the `CSV`
Load the `csv` from `names.csv`. If you want add more you can!

In [ ]:
with open("names.csv") as data_file:
    data = csv.DictReader(data_file)
    data_list = []
    for row in data:
        data_list.append(row)

# Run the script to send the emails!

In [ ]:
context = ssl.create_default_context()
with smtplib.SMTP_SSL(mail_server, port, context=context) as server:
	server.login(user, password)

	for i in data_list:

		receiver_email = i['email']

		response = chat(model='granite3.2', messages= [
		{
			'role': 'user',
			'content': f"You are a professional cold caller for events. You write friendly, engaging emails to help drive people to want to sponsor your events. You use the following information and template email to help write out unique and engaging conversational emails: {template_email}. The date for the event is: {date_of_event}, and if you want to link to the prospectus, it is located here: {prospectus}. Please write an email that convinces me to want to sponsor this event. The recipient's name is {i['name']}. The from is my name as {my_name}, and my email address is {my_email}, my phone number is {my_phone}.",
		},]
	)

		message = MIMEMultipart("alternative")
		message["Subject"] = "Exclusive Sponsorship Opportunity at DevOpsDays Austin 2025"
		message["From"] = sender_email
		message["To"] = receiver_email

		raw_response =  response['message']['content']
		text_email = raw_response

		html_email = markdown.markdown(raw_response)

		part1 = MIMEText(text_email, "plain")
		part2 = MIMEText(html_email, "html")

		message.attach(part1)
		message.attach(part2)

		#print(response['message']['content'])
		server.sendmail(sender_email, receiver_email, message.as_string())
		print(f">>>> email sent to {i['email']} <<<<<")
